In [ ]:
%%capture
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install urduhack

## Data Preprocessing, Tokenizer, Feature Extractor

In [ ]:
from datasets import Dataset
import pandas as pd
import numpy as np
from datasets import load_metric
from urduhack.preprocessing import replace_numbers
from urduhack.preprocessing import normalize_whitespace
from urduhack.preprocessing import remove_english_alphabets
from urduhack.preprocessing import remove_accents
from urduhack.preprocessing import remove_punctuation

Data Preprocessing using Urduhack


*   Train Data



In [ ]:
data = pd.read_csv("/content/drive/MyDrive/NLP_A7_dataset1500 .csv")
data_cleaned = []

for i in data['text']:
  x = remove_punctuation(i)
  x = replace_numbers(i)
  x = normalize_whitespace(i)
  x = remove_english_alphabets(x)
  x = remove_accents(x)

  data_cleaned.append(x)

data['text'] = data_cleaned
data.head()


,key,text
0,/content/drive/MyDrive/21I-2083_Bushra Amjad (...,کہ
1,/content/drive/MyDrive/21I-2083_Bushra Amjad (...,بالکل آپ کی جو ہے وہ
2,/content/drive/MyDrive/21I-2083_Bushra Amjad (...,یہ صرف کمپلین کے لیے یہ آپ
3,/content/drive/MyDrive/21I-2083_Bushra Amjad (...,سر شروع ہو گیا ہےشاید بلڈنگ کا
4,/content/drive/MyDrive/21I-2083_Bushra Amjad (...,پریشانی کا ایشو نہیں ہے ہیلپ لائن آپ



*   Test Data



In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/test_data.csv")
test_data.head()

,key,text
0,/content/drive/MyDrive/Test Dataset/1.wav,جی سر میں ڈیٹیلز چیک کرتا ہوں کائنڈلی ویٹ کیجی...
1,/content/drive/MyDrive/Test Dataset/2.wav,اسلام علیکم کر رہا ہوں جی فرمائے میں آپکی کیا ...
2,/content/drive/MyDrive/Test Dataset/3.wav,میں سَر دیکھ لیتا ہوں جس نمبر سے کال کر رہے ہی...
3,/content/drive/MyDrive/Test Dataset/4.wav,جی جس نمبر سے آپ کال کر رہے ہیں اسی نمبر پہ اش...
4,/content/drive/MyDrive/Test Dataset/5.wav,مزید ویٹ کیجیے گا سر


In [ ]:
import os.path
key = []
text = []

for x in range(len(data['key'])):
  if os.path.isfile(data['key'][x]):
    key.append(data['key'][x])
    text.append(data['text'][x])

data = {'key' : key, 'text' : text}
data = pd.DataFrame(data)
data = Dataset.from_pandas(data)
data


Dataset({
    features: ['key', 'text'],
    num_rows: 1500
})

In [ ]:
key = []
text = []
for x in range(len(test_data['key'])):
  key.append(test_data['key'][x])
  text.append(test_data['text'][x])

test_data = {'key' : key, 'text' : text}
test_data = pd.DataFrame(test_data)
test_data = Dataset.from_pandas(test_data)
test_data

Dataset({
    features: ['key', 'text'],
    num_rows: 169
})

Displaying Random samples

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(data.remove_columns(['key']))

,text
0,ہولڈ کریں
1,السلام علیکم جی اچھا جی نیکسٹ
2,*
3,کھوپرا خوابوں معلوم ہوا کہ وہ پانچ
4,عبدالباری
5,*
6,اب بھی کرن ٹائمنگ نیٹ
7,کلک کرنے کے بعد آپ کے پاس ایک صفہ ہو گا گا
8,کر دیا دیکھیں کوئی جسٹیفیکیشن ہے
9,آپ کی سروسز چیک کی جا رہی انتظار فرماے


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_dict = {
0: ' ',
1: 'ا',
2: 'آ',
3: 'ب',
4: 'پ',
5: 'ت',
6: 'ٹ',
7: 'ث',
8: 'ج',
9: 'چ',
10: 'ح',
11: 'خ',
12: 'د',
13: 'ڈ',
14: 'ذ',
15: 'ر',
16: 'ڑ',
17: 'ز',
18: 'ژ',
19: 'س',
20: 'ش',
21: 'ص',
22: 'ض',
23: 'ط',
24: 'ظ',
25: 'ع',
26: 'غ',
27: 'ف',
28: 'ق',
29: 'ک',
30: 'گ',
31: 'ل',
32: 'م',
33: 'ن',
34: 'ں',
35: 'و',
36: 'ہ',
37: 'ھ',
38: 'ء',
39: 'ئ',
40: 'ی',
41: 'ے'
}

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

44

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

### Create XLSR-Wav2Vec2 Feature Extractor

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

### Preprocess Data

So far, we have not looked at the actual values of the speech signal but just kept the path to its file in the dataset. `XLSR-Wav2Vec2` expects the audio file in the format of a 1-dimensional array, so in the first step, let's load all audio files into the dataset object.

Let's first check the serialization format of the downloaded audio files by looking at the first training sample.

In [ ]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["key"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

In [ ]:
train_set = data.map(speech_file_to_array_fn, remove_columns=data.column_names)
test_set = test_data.map(speech_file_to_array_fn, remove_columns=test_data.column_names)

In [ ]:
import librosa
import numpy as np

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 16_000, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [ ]:
train_set = train_set.map(resample, num_proc=4)
test_set = test_set.map(resample, num_proc=4)

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train_set)-1)

ipd.Audio(data=np.asarray(train_set[rand_int]["speech"]), autoplay=True, rate=16000)

In [ ]:
# rand_int = random.randint(0, len(train_set)-1)

print("Target text:", train_set[rand_int]["target_text"])
print("Input array shape:", np.asarray(train_set[rand_int]["speech"]).shape)
print("Sampling rate:", train_set[rand_int]["sampling_rate"])

Target text: رسید بھائی بانٹیں کیجئے گا اپنا خیال رکھیے گا اللہ حافظ
Input array shape: (34400,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
train_set = train_set.map(prepare_dataset, remove_columns=train_set.column_names, batch_size=8, num_proc=4, batched=True)
test_set = test_set.map(prepare_dataset, remove_columns=test_set.column_names, batch_size=8, num_proc=4, batched=True)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is de

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
train_set

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 1500
})

In [ ]:
test_set

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 169
})

#Training

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/gdrive/MyDrive/wav2vec2-urdu",
  # output_dir="./wav2vec2-large-xlsr-turkish-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [ ]:
data = train_set.train_test_split(test_size=0.2)
data['train']
data['test']

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 300
})

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=data['train'],
    eval_dataset=data['test'],
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1083: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  self.args.max_grad_norm,
/usr/local/lib/python3.7/dist-packages/torch/optim

Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
400,nan,nan,0.949324,16.007200,18.742000
800,nan,nan,0.949324,16.495400,18.187000


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1083: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  self.args.max_grad_norm,
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1083: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  self.args.max_grad_norm,


TrainOutput(global_step=1110, training_loss=nan, metrics={'train_runtime': 2282.1333, 'train_samples_per_second': 0.486, 'total_flos': 2.343129122360832e+18, 'epoch': 29.99, 'init_mem_cpu_alloc_delta': 1680614, 'init_mem_gpu_alloc_delta': 1261935616, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1421040, 'train_mem_gpu_alloc_delta': 3785500672, 'train_mem_cpu_peaked_delta': 35746617, 'train_mem_gpu_peaked_delta': 921328128})

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("/content/gdrive/MyDrive/wav2vec2-urdu/checkpoint-800").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("/content/gdrive/MyDrive/wav2vec2-urdu/checkpoint-800")

In [ ]:
input_dict = processor(test_set[0]["input_values"], return_tensors="pt", padding=True)
logits = model(input_dict.input_values.to("cuda")).logits
pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(test_set["labels"])